In [6]:
# PARTE 3 (Segunda_Parte
# Creacion del Modelo dentro de AWS SageMaker

# Boto3 es el kit de desarrollo de software (SDK) de Amazon Web Services (AWS) para Python
# Boto3 permite al desarrollador de Python escribir software que hace uso de servicios como Amazon S3 y Amazon EC2

import sagemaker
import boto3

In [7]:
# Crear una sesión de sagemaker
sagemaker_session = sagemaker.Session()

# S3 Bucket y prefijo que queremos usar
# default_bucket: crea un bucket de Amazon S3 que se utilizará en esta sesión
bucket = 'repositoriosagemaker'
prefix = 'XGBoost-Regressor'
key = 'XGBoost-Regressor'
# Los roles dan acceso de aprendizaje y al alojamiento a nuestros datos
# Esto se especifica al abrir la instancia de sagemakers en "Crear un rol de IAM"
role = sagemaker.get_execution_role()

In [8]:
print(role)

arn:aws:iam::535002877348:role/service-role/AmazonSageMaker-ExecutionRole-20240927T010873


In [9]:
# leer los datos del archivo csv y luego cargar los datos en el depósito s3
import os

In [11]:
with open('train.csv','rb') as f:
    # El siguiente código carga los datos en el bucket de S3 para acceder más tarde al entrenamiento
    boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train', key)).upload_fileobj(f)

In [12]:
# Imprimamos la ubicación de los datos de entrenamiento en s3
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, prefix, key)
print('ubicación de datos de entrenamiento cargados: {}'.format(s3_train_data))

ubicación de datos de entrenamiento cargados: s3://repositoriosagemaker/XGBoost-Regressor/train/XGBoost-Regressor


In [13]:
with open('validation.csv','rb') as f:
    # El siguiente código carga los datos en el bucket de S3 para acceder más tarde al entrenamiento
    boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation', key)).upload_fileobj(f)

In [14]:
# Imprimamos la ubicación de los datos de validación en s3
s3_validation_data = 's3://{}/{}/validation/{}'.format(bucket, prefix, key)
print('uploaded validation data location: {}'.format(s3_validation_data))

uploaded validation data location: s3://repositoriosagemaker/XGBoost-Regressor/validation/XGBoost-Regressor


In [15]:
# crea un placeholder de salida en el bucket S3 para almacenar la salida

output_location = 's3://{}/{}/output'.format(bucket, prefix)
print('los artefactos de entrenamiento se cargarán en: {}'.format(output_location))

los artefactos de entrenamiento se cargarán en: s3://repositoriosagemaker/XGBoost-Regressor/output


In [16]:
# Este código se usa para obtener el contenedor de entrenamiento de los algoritmos integrados de sagemaker
# todo lo que tenemos que hacer es especificar el nombre del algoritmo que queremos usar

# Obtengamos una referencia a la imagen del contenedor XGBoost

from sagemaker.amazon.amazon_estimator import get_image_uri

container = get_image_uri(boto3.Session().region_name, 'xgboost','0.90-2') # version de XGboost

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [17]:
container

'257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost:0.90-2-cpu-py3'

In [19]:
# Especificar el tipo de instancia que nos gustaría usar para el entrenamiento.
# ruta de salida y sesión de sagemaker en el Estimador.
# También podemos especificar cuántas instancias nos gustaría utilizar para el entrenamiento

# Recordemos que XGBoost funciona combinando un conjunto de modelos débiles para generar resultados precisos / sólidos.
# Los modelos débiles son aleatorios para evitar el sobreajuste

# num_round: el número de rondas para ejecutar el entrenamiento.

# eta: Reducción del tamaño del paso que se utiliza en las actualizaciones para evitar el sobreajuste.


Xgboost_regressor1 = sagemaker.estimator.Estimator(container,
                                       role,
                                       train_instance_count = 1,
                                       train_instance_type = 'ml.m4.xlarge',
                                       output_path = output_location,
                                       sagemaker_session = sagemaker_session)

# Podemos ajustar los hiperparámetros para mejorar el rendimiento del modelo

Xgboost_regressor1.set_hyperparameters(max_depth = 5,
                           objective = 'multi:softmax',
                           num_class=2,
                           #colsample_bytree = 0.3,
                           alpha = 10,
                           eta = 0.5,
                           num_round = 150
                           )

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [20]:
# Creamos los canales "train", "validation" para entregar los datos al modelo
# Fuente: https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-algo-docker-registry-paths.html

train_input = sagemaker.session.s3_input(s3_data = s3_train_data, content_type='csv',s3_data_type = 'S3Prefix')
valid_input = sagemaker.session.s3_input(s3_data = s3_validation_data, content_type='csv',s3_data_type = 'S3Prefix')


data_channels = {'train': train_input,'validation': valid_input}


Xgboost_regressor1.fit(data_channels)

The class sagemaker.session.s3_input has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The class sagemaker.session.s3_input has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-09-27-23-16-00-349


2024-09-27 23:16:00 Starting - Starting the training job...
2024-09-27 23:16:17 Starting - Preparing the instances for training...
2024-09-27 23:16:44 Downloading - Downloading input data...
2024-09-27 23:17:14 Downloading - Downloading the training image...
2024-09-27 23:17:49 Training - Training image download completed. Training in progress..INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value multi:softmax to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
[23:18:02] 22500x33 matrix with 742500 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
INFO:root:Dete

In [ ]:
## DESPLIEGUE EL MODELO PARA REALIZAR LA INFERENCIA

In [21]:
Xgboost_regressor = Xgboost_regressor1.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-09-27-23-26-50-856
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2024-09-27-23-26-50-856
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2024-09-27-23-26-50-856


-----!

In [23]:
import pandas as pd
import numpy as np

In [27]:
# Cargo registro de pruebas que no conocen la variable objetivo
df_xtest = pd.read_csv('xvalidacion.csv', header= None)

In [29]:
X_test = df_xtest.to_numpy()

In [39]:
X_test.shape

(7500, 33)

In [35]:
# El content_type anula los datos que se pasarán al modelo implementado, ya que el modelo implementado espera datos
# en formato texto / csv, lo especificamos como tipo de contenido.

# El serializador acepta un solo argumento, los datos de entrada, y devuelve una secuencia de bytes en el contenido especificado
# tipo

# Referencia: https://sagemaker.readthedocs.io/en/stable/predictors.html

from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer



In [44]:
creditcard_df = pd.read_csv('UCI_Credit_Card_Taiwan.csv')

In [45]:
X_cat = creditcard_df[['SEX', 'EDUCATION', 'MARRIAGE']]

In [46]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder()
X_cat = onehotencoder.fit_transform(X_cat).toarray()

In [47]:
X_cat = pd.DataFrame(X_cat)

In [48]:
X_numerical = creditcard_df[['LIMIT_BAL', 'AGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6',
                'BILL_AMT1','BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6',
                'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']]

In [49]:
X_all = pd.concat([X_cat, X_numerical], axis = 1)

In [50]:
X_all.columns = X_all.columns.astype(str)

In [51]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X_all)

In [52]:
y = creditcard_df['default.payment.next.month']

In [53]:
from sklearn.model_selection import train_test_split
X_train, X_tests, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [61]:
xgb_predictor_reuse=sagemaker.predictor.Predictor(
    endpoint_name="sagemaker-xgboost-2024-09-27-23-26-50-856",
    sagemaker_session=sagemaker.Session(),
    serializer=sagemaker.serializers.CSVSerializer()
)

In [62]:
predictions1 = xgb_predictor_reuse.predict(np.array(X_test))

In [64]:
# código personalizado para convertir los valores en formato de bytes a una matriz

def bytes_2_array(x):

    # realiza la predicción completa como un string y lo divide en función de ','
    l = str(x).split(',')

    # Dado que el primer elemento contiene caracteres no deseados como (b, ',') los eliminamos
    l[0] = l[0][2:]
    # lo mismo que arriba elimina el último carácter no deseado (')
    l[-1] = l[-1][:-1]

    # iterando a través de la lista de cadenas y convirtiéndolas en tipo float
    for i in range(len(l)):
        l[i] = float(l[i])

    # convertimos la lista a array
    l = np.array(l).astype('float32')

    # redimensionamos la matriz unidimensional a una matriz bidimensional
    return l.reshape(-1,1)

In [65]:
predicted_values = bytes_2_array(predictions1)

In [66]:
predicted_values

array([[0.],
       [1.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

In [67]:
predicted_values.shape

(7500, 1)

In [69]:
y_test_ar = np.array(y_test)
y_test_ar = y_test_ar.reshape(-1,1)

In [71]:
y_test_ar.shape

(7500, 1)

In [76]:
from sklearn.metrics import precision_score, recall_score, accuracy_score

print("Precision = {]",format(precision_score(y_test_ar, predicted_values,average='macro')))
print("Recall = {]",format(recall_score(y_test_ar, predicted_values,average='macro')))
print("Accuracy = {]",format(accuracy_score(y_test_ar, predicted_values)))

Precision = {] 0.4979914278586219
Recall = {] 0.49863060648051793
Accuracy = {] 0.6982666666666667
